## Análisis de Tarifas y Costos de Energía Eléctrica para el Mercado Regulado

Este conjunto de datos fue extraído del portal de **Datos Abiertos de Colombia**, y corresponde a la información de tarifas y costos de energía eléctrica para usuarios del mercado regulado, específicamente del archivo *Tarifas_y_Costos_de_Energía_Eléctrica_para_el_Mercado_Regulado_20250429*.

### Descripción del conjunto de datos

Cada fila representa el costo de prestación del servicio eléctrico a un usuario residencial o comercial, detallando distintos componentes que conforman el precio total que paga el consumidor. El dataset contiene los siguientes atributos numéricos:

- `CU Total`: Costo unitario total de la energía por kilovatio hora ($/kWh).
- `Costo Compra (Gm,i)`: Costo de compra de energía.
- `Cargo Transporte STN (Tm)`: Costo de transporte en el Sistema de Transmisión Nacional.
- `Cargo Transporte SDL (Dn,m)`: Costo de transporte en el Sistema de Distribución Local.
- `Margen Comercialización (CVm,i,j)`: Margen para cubrir costos de comercialización.
- `Costo G, T, Pérdidas (PRn,m)`: Costo total de generación, transporte y pérdidas.
- `Restricciones (Rm)`: Costo asociado a restricciones operativas en la red.
- `Cfm,j ($/fact.)`: Costo fijo mensual por factura.
- `COT`: Costo total facturado.

### Objetivo del análisis

Vamos a construir un árbol KD (k-d tree) utilizando como variables de entrada los atributos numéricos anteriores, y definiremos una **clasificación multiclase** según el valor de `CU Total`, que será nuestra **variable objetivo (target)**.

Para ello, crearemos 3 clases:

- **Clase 0**: `CU Total` bajo (menor o igual al percentil 33).
- **Clase 1**: `CU Total` medio (entre el percentil 33 y el percentil 66).
- **Clase 2**: `CU Total` alto (mayor al percentil 66).

Esta transformación convierte el problema en una tarea de clasificación supervisada basada en rangos del costo total unitario. Esto nos permitirá construir reglas de decisión y verificar el rendimiento del árbol KD en esta clasificación.


In [24]:
import pandas as pd
import numpy as np

# Cargar el dataset
df = pd.read_csv("Dataset.csv")

# Limpiar nombres de columnas (quitar espacios extra)
df.columns = df.columns.str.strip()

# Calcular percentiles para clasificar 'CU Total'
p33 = np.percentile(df["CU Total"], 33)
p66 = np.percentile(df["CU Total"], 66)

# Crear función de clasificación
def clasificar(cu):
    if cu <= p33:
        return 0
    elif cu <= p66:
        return 1
    else:
        return 2

# Aplicar clasificación
df["CU Total"] = df["CU Total"].apply(clasificar)

# Guardar nuevo dataset con clases
df.to_csv("Dataset_clases.csv", index=False)

# Mostrar primeros registros para verificar
df[["CU Total", "Costo Compra (Gm,i)",	"Cargo Transporte STN (Tm)",	"Cargo Transporte SDL (Dn,m)",	"Margen Comercialización (CVm,i,j)",	"Costo G, T, Pérdidas (PRn,m)",	"Restricciones (Rm)",	"Cfm,j ($/fact.)",	"COT"
]].head(10)


,CU Total,"Costo Compra (Gm,i)",Cargo Transporte STN (Tm),"Cargo Transporte SDL (Dn,m)","Margen Comercialización (CVm,i,j)","Costo G, T, Pérdidas (PRn,m)",Restricciones (Rm),"Cfm,j ($/fact.)",COT
0,1,289.09,54.32,256.98,70.27,58.26,18.49,9286.0,66.32
1,1,289.09,54.32,234.24,70.27,58.26,18.49,9286.0,10.83
2,1,289.09,54.32,211.50,70.27,58.26,18.49,9286.0,42.66
3,0,289.09,54.32,158.35,70.27,19.73,18.49,9286.0,83.74
4,0,289.09,54.32,110.75,70.27,19.30,18.49,9286.0,34.96
5,2,289.09,54.32,261.91,112.06,57.36,19.21,15291.0,71.35
6,2,289.09,54.32,225.50,112.06,57.36,19.21,15291.0,69.42
7,1,289.09,54.32,189.09,112.06,57.36,19.21,15291.0,76.51
8,0,289.09,54.32,157.72,112.06,24.73,19.21,15291.0,48.39
9,0,289.09,54.32,94.45,112.06,21.23,19.21,15291.0,31.05


## 🌳 Construcción del Árbol KD
En este notebook se construye un Árbol KD para clasificar registros del mercado regulado de energía en Colombia, usando variables como costos de compra, transporte, pérdidas, comercialización, entre otros. A continuación se describe paso a paso el proceso de construcción del árbol, con un ejemplo práctico usando datos reales del dataset.

---

### 📌 1. ¿Qué es un Árbol KD?

El Árbol KD (k-d tree) es una estructura binaria que organiza datos en un espacio multidimensional. En lugar de dividir con reglas fijas como en un árbol de decisión tradicional, un KD-Tree:

- Divide usando **atributos numéricos rotativos** (por eje).
- Usa la **mediana** del atributo actual como umbral.
- Crea ramas izquierda y derecha con base en ese umbral.

Esto permite construir una estructura eficiente para tareas de clasificación o búsqueda, especialmente en datos con muchas variables numéricas.

---

### 🔁 2. Proceso de Construcción Recursiva

Para construir el árbol:

1. **Se elige un eje de división**:  
   Usamos `axis = depth % k`, donde `depth` es la profundidad del nodo actual y `k` es el número total de atributos. Esto asegura que se roten los ejes (columnas) en cada nivel.

2. **Se ordenan los datos por ese eje** y se escoge la mediana como nodo raíz.

3. **Se dividen los datos** en dos grupos:
   - Izquierda: valores menores o iguales al umbral (mediana).
   - Derecha: valores mayores.

4. **Se repite recursivamente** para cada grupo, creando nuevos nodos.

---

### 🧪 3. Ejemplo con Datos Reales (16 registros)

A continuación mostramos una porción del dataset (atributo objetivo: `CU Total`, convertido en clases 0, 1 o 2):

| CU Total | Costo Compra | STN | SDL | Comercialización | Pérdidas | Restricciones | Costo Fijo | COT |
|----------|--------------|-----|-----|------------------|----------|---------------|------------|-----|
| 1 | 289.09 | 54.32 | 256.98 | 70.27 | 58.26 | 18.49 | 9286.0 | 66.32 |
| 1 | 289.09 | 54.32 | 234.24 | 70.27 | 58.26 | 18.49 | 9286.0 | 10.83 |
| 1 | 289.09 | 54.32 | 211.50 | 70.27 | 58.26 | 18.49 | 9286.0 | 42.66 |
| 0 | 289.09 | 54.32 | 158.35 | 70.27 | 19.73 | 18.49 | 9286.0 | 83.74 |
| 0 | 289.09 | 54.32 | 110.75 | 70.27 | 19.30 | 18.49 | 9286.0 | 34.96 |
| 2 | 289.09 | 54.32 | 261.91 | 112.06 | 57.36 | 19.21 | 15291.0 | 71.35 |
| 2 | 289.09 | 54.32 | 225.50 | 112.06 | 57.36 | 19.21 | 15291.0 | 69.42 |
| 1 | 289.09 | 54.32 | 189.09 | 112.06 | 57.36 | 19.21 | 15291.0 | 76.51 |
| 0 | 289.09 | 54.32 | 157.72 | 112.06 | 24.73 | 19.21 | 15291.0 | 48.39 |
| 0 | 289.09 | 54.32 | 94.45 | 112.06 | 21.23 | 19.21 | 15291.0 | 31.05 |
| 2 | 289.09 | 54.32 | 256.98 | 93.08 | 83.20 | 19.21 | 9594.0  | 50.03 |
| 1 | 289.09 | 54.32 | 204.74 | 93.08 | 83.20 | 19.21 | 9594.0  | 47.47 |
| 1 | 289.09 | 54.32 | 152.50 | 93.08 | 83.20 | 19.21 | 9594.0  | 57.59 |
| 0 | 289.09 | 54.32 | 158.35 | 93.08 | 36.15 | 19.21 | 9594.0  | 70.61 |
| 0 | 289.09 | 54.32 | 110.75 | 93.08 | 26.85 | 19.21 | 9594.0  | 60.77 |
| 1 | 289.09 | 54.32 | 261.91 | 66.18 | 56.80 | 18.10 | 8565.0  | 46.19 |

---

### 🔍 4. Ejemplo de División Real

#### Nivel 0 (raíz):
- Eje: **`Costo Compra (Gm,i)`**
- Todos los valores son iguales: **289.09**
- → El algoritmo pasa al siguiente eje.

#### Nivel 1:
- Eje: **`Cargo Transporte SDL (Dn,m)`**
- Ordenamos por este eje y seleccionamos la mediana (~158.35)
- Nodo raíz: valor `[158.35]` → se divide:
  - Izquierda: valores SDL ≤ 158.35
  - Derecha: valores SDL > 158.35

#### Nivel 2:
- Se repite el proceso con el siguiente atributo (`Margen Comercialización`, luego `PRn,m`, etc.)

---

### 🧠 5. Resultado: ¿Qué representa el árbol?

- Cada nodo representa una **regla de decisión basada en un atributo**.
- La ruta desde la raíz hasta una hoja representa una **combinación de condiciones** que conducen a una clase (nivel bajo, medio o alto de `CU Total`).
- Este árbol permite visualizar cómo se segmentan los usuarios en base a los componentes tarifarios de su servicio.

---

### 📈 6. Visualización

El árbol fue visualizado usando la librería `pyvis`, generando un grafo interactivo jerárquico que puede ser explorado para observar:

- Qué atributos se usan para dividir.
- Qué valores sirven como umbral.
- A qué clase pertenece cada nodo hoja.

---

### ✅ Conclusión

Este Árbol KD nos permite clasificar de manera explicativa a los usuarios del mercado eléctrico, entendiendo qué factores están más relacionados con un `CU Total` bajo, medio o alto, y visualizando decisiones de clasificación basadas en los atributos tarifarios.



In [25]:
import numpy as np

class KDNode:
    def __init__(self, point, label, axis, left=None, right=None):
        self.point = point  # coordenadas (atributos)
        self.label = label  # clase
        self.axis = axis    # eje de división
        self.left = left
        self.right = right

def construir_kd_tree(data, labels, depth=0):
    n = len(data)
    if n == 0:
        return None

    # Determinar eje de división (rota entre atributos)
    k = data.shape[1]
    axis = depth % k

    # Ordenar por el eje y dividir por la mediana
    sorted_indices = data[:, axis].argsort()
    data = data[sorted_indices]
    labels = labels[sorted_indices]
    med = n // 2

    return KDNode(
        point=data[med],
        label=labels[med],
        axis=axis,
        left=construir_kd_tree(data[:med], labels[:med], depth + 1),
        right=construir_kd_tree(data[med + 1:], labels[med + 1:], depth + 1)
    )


## Visualización del Árbol KD

Para interpretar la estructura del árbol KD de forma clara y comprensible, usaremos la biblioteca `pyvis` para generar una visualización interactiva en HTML. Cada nodo del árbol será representado como un círculo, con líneas que conectan a sus nodos hijo.

- **Cada nodo** mostrará:
  - El valor del punto (atributos del ejemplo).
  - El eje de división (X₀, X₁, X₂, ...).
  - La clase asignada al punto.
- **Las conexiones** representan cómo se divide recursivamente el conjunto de datos.
- El árbol puede visualizarse fácilmente como una jerarquía descendente, lo que permite identificar reglas de decisión y patrones de clasificación.

Este enfoque facilita la explicación y análisis de las divisiones que hace el árbol KD para separar las clases.


In [26]:
import pandas as pd
import numpy as np
from pyvis.network import Network
from IPython.display import IFrame

# ---------------------------
# Estructura del nodo KD
# ---------------------------
class KDNode:
    def __init__(self, point, label, axis, left=None, right=None):
        self.point = point
        self.label = label
        self.axis = axis
        self.left = left
        self.right = right

# ---------------------------
# Construcción recursiva
# ---------------------------
def construir_kd_tree(data, labels, depth=0):
    if len(data) == 0:
        return None

    k = data.shape[1]
    axis = depth % k

    sorted_indices = data[:, axis].argsort()
    data = data[sorted_indices]
    labels = labels[sorted_indices]
    med = len(data) // 2

    return KDNode(
        point=data[med],
        label=labels[med],
        axis=axis,
        left=construir_kd_tree(data[:med], labels[:med], depth + 1),
        right=construir_kd_tree(data[med + 1:], labels[med + 1:], depth + 1)
    )

def agregar_nodos_aristas(net, node, parent_id=None, node_id=0, depth=0, columnas=None):
    if node is None:
        return node_id

    # Obtener el nombre del atributo que se usó para dividir
    nombre_atributo = columnas[node.axis] if columnas else f"X{node.axis}"
    umbral = round(node.point[node.axis], 2)

    # Crear etiqueta del nodo
    label = f"Clase: {node.label}\nEje: {nombre_atributo}\nUmbral: {umbral}"
    current_id = node_id
    net.add_node(current_id, label=label, title=label, level=depth)

    if parent_id is not None:
        net.add_edge(parent_id, current_id)

    node_id += 1
    node_id = agregar_nodos_aristas(net, node.left, current_id, node_id, depth + 1, columnas)
    node_id = agregar_nodos_aristas(net, node.right, current_id, node_id, depth + 1, columnas)

    return node_id

def visualizar_arbol_kd(root, columnas, output_file="arbol_kd.html"):
    net = Network(height="800px", width="100%", directed=True)

    net.set_options("""
    {
      "layout": {
        "hierarchical": {
          "enabled": true,
          "levelSeparation": 120,
          "nodeSpacing": 100,
          "treeSpacing": 200,
          "direction": "UD",
          "sortMethod": "directed"
        }
      },
      "physics": {
        "hierarchicalRepulsion": {
          "nodeDistance": 120
        },
        "solver": "hierarchicalRepulsion"
      }
    }
    """)

    agregar_nodos_aristas(net, node=root, columnas=columnas)
    net.save_graph(output_file)



# ---------------------------
# Cargar datos y construir árbol
# ---------------------------
df = pd.read_csv("Dataset_clases.csv")
df.columns = df.columns.str.strip()

# Asignar X (atributos) y y (clase)
X = df.drop(columns=["CU Total", "COT", "Clase"], errors='ignore')  # quitar la clase y columnas dependientes
y = df["CU Total"] if "Clase" not in df.columns else df["Clase"]

# Asegurarse de que X es solo numérico
X = X.select_dtypes(include=[np.number])

# Columnas del dataset para mostrar nombres de atributos
columnas = X.columns.tolist()

# Construir y visualizar árbol KD
root = construir_kd_tree(X.values, y.values)
visualizar_arbol_kd(root, columnas)



## 🧠 Extracción de Reglas desde un Árbol KD

Una vez construido el Árbol KD, podemos transformarlo en un conjunto de **reglas de decisión interpretables**. Para ello, implementamos un algoritmo que **recorre el árbol desde la raíz hasta cada nodo hoja**, generando reglas del tipo: Si [condición 1] y [condición 2] y ... entonces clase = [etiqueta]


---

### ⚙️ ¿Cómo funciona el algoritmo de extracción?

La función `extraer_reglas` sigue una estrategia **recursiva** y consiste en:

1. **Iniciar desde la raíz del árbol.**
2. En cada nodo:
   - Obtener el **atributo (eje)** que se usó para dividir.
   - Obtener el **valor umbral** de ese atributo (la coordenada del punto del nodo en el eje actual).
3. **Construir dos ramas**:
   - Una para el caso donde el valor del atributo es `≤ umbral` → se va al hijo izquierdo.
   - Otra para el caso donde el valor del atributo es `> umbral` → se va al hijo derecho.
4. **Acumular las condiciones** (como texto) en una lista a medida que se desciende por el árbol.
5. **Al llegar a una hoja**, concatenar todas las condiciones acumuladas y registrar la clase del nodo hoja.
6. **Guardar la regla final** en una lista.

Este procedimiento se repite recursivamente para cada rama del árbol, generando una regla distinta por cada hoja.

---

### 🧾 Ejemplo de una regla generada

Supongamos que un recorrido por el árbol sigue estas decisiones:

- Cargo Transporte SDL (Dn,m) ≤ 158.35  
- Margen Comercialización (CVm,i,j) > 26.85  
- Restricciones (Rm) ≤ 19.21  

Y que la hoja a la que llega tiene clase `1`. Entonces la regla generada será: Si Cargo Transporte SDL (Dn,m) ≤ 158.35 y Margen Comercialización (CVm,i,j) > 26.85 y Restricciones (Rm) ≤ 19.21 entonces clase = 1


---

### 📌 Detalles técnicos

- Cada nodo guarda:
  - `axis`: índice del atributo usado como eje de división.
  - `point`: vector de valores numéricos del punto actual.
  - `label`: clase asignada si es un nodo hoja.

---




In [27]:
def extraer_reglas(node, columnas, condiciones=None, reglas=None):
    if condiciones is None:
        condiciones = []
    if reglas is None:
        reglas = []

    if node is None:
        return reglas

    # Si es una hoja (sin hijos)
    if node.left is None and node.right is None:
        regla = " y ".join(condiciones)
        reglas.append(f"Si {regla} entonces clase = {node.label}")
        return reglas

    atributo = columnas[node.axis]
    umbral = round(node.point[node.axis], 2)

    # Rama izquierda: valor <= umbral
    condiciones_izq = condiciones + [f"{atributo} ≤ {umbral}"]
    extraer_reglas(node.left, columnas, condiciones_izq, reglas)

    # Rama derecha: valor > umbral
    condiciones_der = condiciones + [f"{atributo} > {umbral}"]
    extraer_reglas(node.right, columnas, condiciones_der, reglas)

    return reglas


In [28]:
# Extraer reglas desde el árbol
reglas = extraer_reglas(root, columnas)

# Guardarlas en un archivo .txt
with open("reglas_kd_tree.txt", "w", encoding="utf-8") as f:
    for i, regla in enumerate(reglas, 1):
        f.write(f"Regla {i}: {regla}\n")


## ✅ Verificación de la Proporción de Acierto por Reescritura

Una vez construido el Árbol KD, evaluamos qué tan bien es capaz de **clasificar los mismos datos con los que fue entrenado**. A este proceso se le conoce como **reescritura**, y su objetivo es validar que las reglas extraídas del árbol son coherentes con las clases originales del dataset.

---

### 🧠 ¿Qué se hizo?

1. Se recorrió el Árbol KD desde la raíz hasta las hojas, aplicando la lógica de división en cada nodo:
   - Si el valor del atributo correspondiente al eje (`axis`) es menor o igual al umbral (`point[axis]`), se va a la rama izquierda.
   - Si es mayor, se va a la rama derecha.
2. Al llegar a una **hoja**, se devuelve la clase almacenada en ese nodo como **predicción**.
3. Este proceso se repite para **cada fila del dataset de entrenamiento**.
4. Finalmente, se compara cada predicción con la clase real y se calcula el **porcentaje de aciertos (accuracy)**.

---

---

### 📈 Resultado esperado

Después de ejecutar el código, se imprime una salida como la siguiente:



In [30]:
from sklearn.metrics import accuracy_score

# ----------------------------
# Función para predecir con el árbol KD
# ----------------------------
def predecir_kd_tree(x, node):
    """
    Recorre el árbol KD para predecir la clase de un punto x.
    """
    while node is not None and (node.left or node.right):
        if x[node.axis] <= node.point[node.axis]:
            node = node.left
        else:
            node = node.right
    return node.label if node else None

# ----------------------------
# Realizar predicciones sobre el mismo conjunto de entrenamiento
# ----------------------------
y_pred = [predecir_kd_tree(x, root) for x in X.values]

# ----------------------------
# Filtrar solo las predicciones válidas (evita errores por nodos vacíos)
# ----------------------------
validez = [pred is not None for pred in y_pred]
y_validos = [yi for yi, v in zip(y, validez) if v]
y_pred_validos = [yp for yp in y_pred if yp is not None]

# ----------------------------
# Calcular proporción de acierto (accuracy de reescritura)
# ----------------------------
accuracy = accuracy_score(y_validos, y_pred_validos)
print(f"Proporción de acierto por reescritura: {accuracy:.2%}")


Proporción de acierto por reescritura: 80.52%
